In [ ]:
#!usr/bin/env python

######################################## Format argparse ##########################################################
# 1. Write script parameters to take in the four files as well as the index.tsv file and a min quality score cut off.

import argparse
import numpy as np

parser = argparse.ArgumentParser(description="Quality Index Swapping")
parser.add_argument("-f1", "--filepath1", help="R1 file", required=True, type=str)
parser.add_argument("-f2", "--filepath2", help="R2 file", required=True, type=str)
parser.add_argument("-f3", "--filepath3", help="R3 file", required=True, type=str)
parser.add_argument("-f4", "--filepath4", help="R4 file", required=True, type=str)
parser.add_argument("-f5", "--filepath5", help="Indexes.txt file", required=True, type=str)
parser.add_argument("-min_qscore", "--min_cutoff", help="Min quality score cutoff", required=True, type=int)


args = parser.parse_args() #sets arguments as call-able 
f1 = args.filepath1
f2 = args.filepath2
f3 = args.filepath3
f4 = args.filepath4
f5 = args.filepath5
min_qscore = args.min_cutoff


def convert_phred(letter):
    n = ord(letter) - 33
    return n


######################## Build dictionary containing every 4 nucleotide seq combination ############################

# Every possible combo of 2, 8 base long indexes as key and value = counter(increases by 1 every time seq is 
# uncovered in index reads R2, R3)
N_Idx_Dic = {}

index_values={} #dictionary to hold all combinations of index values from said FASTQ/Index files
list_of_index_reads=[]#list holding index reads from said index file
un_matched_index = {} #list that has index reads 



            
with open(f5) as tsv_fh: #open tsv file, set to filehandle (fh)
    for line in tsv_fh:
        values_per_column=line.strip().split("\t") #strip and split by tab for each value in both columns
        index_reads=(values_per_column[4]) #grab index reads
        list_of_index_reads.append(index_reads) #put into list_of_index_reads array
    #print(list_of_index_reads)

#generate all possible combonations of 2 index values and store in dict with value of 0
for line_in_file in list_of_index_reads:
    for indexes in list_of_index_reads:
        if line_in_file == indexes:
            index_values[line_in_file + "_" + indexes]=0#set indexes to 0 [index1,index2:0]
        else:
            un_matched_index[line_in_file + "_" + indexes] = 0  #Generate underscore between each line for indexes in dic        


############################################### Setting variables/read in files ##########################################################

head1 = "" #setting header variable for each corresponding first line in each read of the FASTQ file
head2 = ""
head3 = ""
head4 = ""

seq1 = "" #setting sequence variable for each corresponding second line in each read of the FASTQ file
seq2 = ""
seq3 = ""
seq4 = ""

plus_symbol1 = "" #setting (+) symbol to variable for each corresponding third line in each read of the FASTQ file
plus_symbol2 = ""
plus_symbol3 = ""
plus_symbol4 = ""

q_score1 = "" #setting qscore variable for each corresponding fourth line in each read of the FASTQ file
q_score2 = ""
q_score3 = ""
q_score4 = ""



############Setting global vairables for each said counter below############
N_counter = 0
matched_counter = 0
idx_hop_counter = 0
sequencing_error_counter = 0
retained_values_counter = 0
discarded_values_counter = 0
total_counter = 0




def rc(dna):
    sequence_dictionary = {'A':'T','T':'A','G':'C','C':'G', 'N':'N'}
    return "".join([sequence_dictionary[base] for base in reversed(dna)])
 

    
    
    
##################################### Assigning each line in FASTQ file to a variable ######################################
    
    
with open(f1, "rt") as fh_R1, open(f2, "rt") as fh_R2, open(f3) as fh_R3, open(f4) as fh_R4:
    for line in fh_R1:
        line = line.strip("\n")
        head1 = line.strip("\n")
        seq1 = fh_R1.readline().strip("\n")
        plus_symbol1 = fh_R1.readline().strip("\n")
        q_score1 = fh_R1.readline().strip("\n")

        head2 = fh_R2.readline().strip("\n")
        seq2 = fh_R2.readline().strip("\n")
        plus_symbol2 = fh_R2.readline().strip("\n")
        q_score2 = fh_R2.readline().strip("\n")

        head3 = fh_R3.readline().strip("\n")
        seq3 = rc(fh_R3.readline().strip("\n"))
        plus_symbol3 = fh_R3.readline().strip("\n")
        q_score3 = fh_R3.readline().strip("\n")

        head4 = fh_R4.readline().strip("\n")
        seq4 = fh_R4.readline().strip("\n")
        plus_symbol4 = fh_R4.readline().strip("\n")
        q_score4 = fh_R4.readline().strip("\n")

        value_of_index = seq2 + "_" + seq3

        total_counter += 1
        
        if total_counter%500000000:
            print("On line: ", total_counter, flush = True)

        raw_score = []
        for base in range(len(q_score2)):        
            raw_score.append(convert_phred(q_score2[base]))
            raw_score.append(convert_phred(q_score3[base]))
        
        if all(items >= min_qscore for items in raw_score):  
            retained_values_counter+=1 
            if value_of_index in index_values:           
                index_values[value_of_index] += 1
                matched_counter += 1
            
            if value_of_index in un_matched_index:    
                un_matched_index[value_of_index] += 1
                idx_hop_counter += 1
            
            if "N" in value_of_index:
                if value_of_index in N_Idx_Dic:
                    N_Idx_Dic[value_of_index]+=1
                    N_counter += 1
                else:
                    N_Idx_Dic[value_of_index]=1
                    N_counter += 1
            
            if value_of_index not in index_values and value_of_index not in un_matched_index and value_of_index not in N_Idx_Dic:
                sequencing_error_counter += 1      
    
        else:
            discarded_values_counter += 1 
            
##################################### Outputting stats and dictionary file ############################################
            
            
with open("1294_S1_L008_R"+ "_Output_table_cov_0", 'w') as output1:
    j = 0
    m = 0   
    k = 0
    output1.write("\n" + "Number of Undetermined N Reads: " + str(N_counter))
    output1.write("\n" + "Number of Expected Paired Index Reads : " + str(matched_counter))
    output1.write("\n" + "Number of Index Swapped Reads: " + str(idx_op_counter))
    output1.write("\n" + "Number of Sequencing Error Reads: " + str(sequencing_error_counter))
    output1.write("\n" + "Number of Retained Reads: " + str(retained_values_counter))
    output1.write("\n" + "Number of Discarded Reads: " + str(discarded_values_counter))
    output1.write("\n" + "Total Number of Reads: " + str(total_counter))
    output1.write("\n" + "Percentage of Reads Index Swapped: " + str((idx_op_counter/total_counter)*100))
    output1.write("\n" + "Percentage of Expected Paired Index Reads: " + str((matched_counter/total_counter)*100))
    output1.write("\n")
    output1.write("min_qscore: " + min_qscore + "\n")
    output1.write("\n")
    output1.write("Dual Indexed Pairs" + "\t" + "Counts" + "\n")
    
    for i in index_values:
        output1.write(str(i) + "\t" + str(index_values[i]) + "\n")
        j+=1       
    output1.write("\n")
    output1.write(("Index Hopped Pairs" + "\t" + "Counts" + "\n"))
    for i in un_matched_index:
        output1.write(str(i) + "\t" + str(un_matched_index[i]) + "\n")
        m+=1
    output1.write("\n")
    output1.write(("N Indexed Pairs" + "\t" + "Counts" + "\n"))
    for i in N_Idx_Dic:
        output1.write(str(i) + "\t" + str(N_Idx_Dic[i]) + "\n")
        k+=1       
   